# PRE-PROCESSING:

In [43]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [44]:
def importa_dati(path, sep=','):
    """Funzione per l'importazione di un dataset con un formato csv in un DataFrame di Pandas.
       - path: percorso in cui è salvato il file.csv (stringa);
       - sep: separatore utilizzato nel file (stringa)"""
    data = pd.read_csv(path, sep=sep)
    data.drop('Unnamed: 0', axis=1, inplace=True)
    return data

In [45]:
# nome dataset da importare
df_name = 'bin'
# tipo di pre-processing
pre_type = 'le'

df = importa_dati('/Users/eliaceccolini/Documents/Uni/Tesi/Dataset_finale/Datasets/'+df_name+'.csv')

# nome nuovo dataset da esportare
export_name = df_name+'_'+pre_type

***
### Funzione per categorizzare il dataset

In [46]:
def categorizza(data, ranges):
    """Funzione per categorizzare le variabili numeriche continue del dataset.
       - data: dataset (DataFrame)
       - ranges: intervalli per ogni paziente per categorizzare i valori dei prelievi """
    
    # colonne da dividere in categorie
    cols_bmi = ['somm'+str(i)+'_BMI' for i in range(1,7)]
    cols_val = []
    for n in ['creatinina', 'creatinina2', 'glucosio', 'emoglobina']:
        cols_val = cols_val + ['somm'+str(i)+'_'+n for i in range(1,7)]

    # divisione categorie BMI
    for col in cols_bmi:
        for i in range(data.shape[0]):
            if data.loc[(i, col)] <= 18.4:
                data.loc[(i, col)] = 'sottopeso'
            elif data.loc[(i, col)] <= 24.9:
                data.loc[(i, col)] = 'normo-peso'
            elif data.loc[(i, col)] <= 29.9:
                data.loc[(i, col)] = 'sovrappeso'
            else:
                data.loc[(i, col)] = 'obeso'
            
    # divisione categorie valori prelievi
    for i,col in enumerate(cols_val):
        curr = ranges[i]
        for i in range(data.shape[0]):
            if (data.loc[(i, col)] < curr[i,0]):
                data.loc[(i, col)] = 'underRange'
            elif (data.loc[(i, col)] > curr[i,1]):
                data.loc[(i, col)] = 'overRange'
            else:
                data.loc[(i, col)] = 'inRange'

***
### Funzione per normalizzare il dataset

In [47]:
def normalizza(data):
    """Funzione per normalizzare le variabili numeriche continue del dataset.
       - data: dataset (DataFrame)"""
    
    # nomi colonne da normalizzare
    cols_bmi = ['somm'+str(i)+'_BMI' for i in range(1,7)]
    name_cols_val = ['creatinina', 'creatinina2', 'glucosio', 'emoglobina']
    cols_val = []
    for n in name_cols_val:
        cols_val = cols_val + ['somm'+str(i)+'_'+n for i in range(1,7)]
    cols = cols_bmi + cols_val
    
    scaler = MinMaxScaler()
    data[cols] = pd.DataFrame(scaler.fit_transform(X=data[cols]), dtype=float)

***
### Trasformazione dei dati categoriali:

In [48]:
def applica_OHE(data, cols):
    """Funzione per trasformare le colonne in tipo categoriale codificandole tramite OHE.
       - data: dataset (DataFrame)
       - cols: nome colonne da trasformare (lista di String) """
    
    # creazione One Hot Encoder
    OHenc = OneHotEncoder(handle_unknown='ignore', sparse_output=False, feature_name_combiner='concat')
    
    # applicazione OHE
    new_cat_data = pd.DataFrame(OHenc.fit_transform(data[cols]), dtype=int)
    new_cat_data.columns = OHenc.get_feature_names_out()
    new_cat_data['index'] = new_cat_data.index
    data.drop(cols, axis=1, inplace=True)
    data['index'] = data.index
    new_data = pd.merge(data, new_cat_data, on='index')
    new_data.drop('index', axis=1, inplace=True)
    #null_cols = [col for col in new_data.columns if 'nan' in col]
    #new_data.drop(null_cols, axis=1, inplace=True)
    
    return new_data

In [49]:
def applica_LE(data, cols):
    """Funzione per trasformare le colonne in tipo categoriale codificandole tramite LE.
       - data: dataset (DataFrame)
       - cols: nome colonne da trasformare (lista di String) """
    
    for col in cols:
        data[col] = data[col].astype('category').cat.codes
    return data

In [50]:
def trasforma_categoriali(data, t, cat=False, ranges=None, norm=False):
    """Funzione per trasformare le colonne in tipo categoriale codificandole tramite OHE.
       - data: dataset (DataFrame)
       - t: tipo di trasformazione (String - o=ohe, l=le)
       - cat: valore che indica se categorizzare le variabili numeriche continue (Boolean)
       - ranges: intervalli per categorizzare i valori dei prelievi (array)
       - norm: valore che indica se normalizzare le variabili numeriche continue (Boolean) """
    
    # restituzione errore se sono True sia la categorizzazione che la normalizzazione
    if cat and norm:
        raise Exception("Errore! E' possibile applicare solamente una tra la categorizzazione e la normalizzazione.")
        
    # elenco nome colonne categoriali
    cols_cat = ['sesso', 'consumatore_alcool', 'fumatore', 'gruppo_patologia', 'classe_eta', 'classi_principi_attivi', 't', 'n', 'm']
    sympt_names = ['fatigue', 'pain', 'nausea', 'diarrea']
    cols_sympt = []
    for name in sympt_names:
        cols_sympt = cols_sympt + ['somm'+str(i)+'_'+name for i in range(1, 7)]
    cols_cat = cols_cat + cols_sympt
        
    # categorizzazione BMI e valori prelievi se sono da categorizzare la var numeriche continue
    if cat:
        categorizza(data, ranges)
        cols_bmi = ['somm'+str(i)+'_BMI' for i in range(1,7)]
        values_names = ['creatinina', 'creatinina2', 'glucosio', 'emoglobina']
        cols_val = []
        for n in values_names:
            cols_val = cols_val + ['somm'+str(i)+'_'+n for i in range(1,7)]
        cols_cat = cols_cat + cols_bmi + cols_val
        
    # normalizzazione del BMI e valori prelievi se sono da normalizzare le var numeriche continue
    if norm:
        normalizza(data)
    
    if t == 'o':
        new_data = applica_OHE(data, cols_cat)
    else:
        new_data = applica_LE(data, cols_cat)
    
    return new_data, set(data.columns) - set(cols_cat) - set(['index'])

In [51]:
df, cols_num = trasforma_categoriali(df, 'l')

***
### Creazione valori prelievi

In [52]:
def crea_valori(data, col):
    """Funzione per sostiruire le stringhe nei valori dei prelievi con il valore corretto.
       - data: dataset (DataFrame)
       - cols: nome colonna da elaborare (String) """
    
    # allocazione memoria per salvare gli estremi di tutti gli intervalli
    ranges = np.zeros((data.shape[0], 2))
    
    data[col] = [str(n).replace(",", ".") for n in data[col]]
    
    for i in range(data.shape[0]):
        if data.loc[(i, col)] != str(np.nan):
            l = data.loc[(i, col)].split(' ')
            try:
                val = float(l[0])
                minR = float(l[2].replace('(', ''))
                maxR = float(l[4].replace(')', ''))
            except ValueError:
                val = np.nan
            data.loc[(i, col)] = val
            ranges[i] = [minR, maxR]
    
    data[col] = [float(p) for p in data[col]]
    mean = np.mean(data[col].dropna())
    
    return ranges

In [53]:
# elenco nomi colonne
name_cols = ['creatinina', 'creatinina2', 'glucosio', 'emoglobina']
cols = []
for n in name_cols:
    cols = cols + ['somm'+str(i)+'_'+n for i in range(1,7)]

# allocazione memoria dei range
ranges = np.zeros((len(cols)), dtype=np.ndarray)

# creazione valori
for i,col in enumerate(cols):
    r = crea_valori(df, col)
    ranges[i] = r

***
### Sostituzione del peso e dell'altezza con BMI:

In [54]:
def calcola_BMI(data, colonna_peso, colonna_alt, nome_colonna_bmi):
    """Funzione per il calcolo del BMI. Inoltre le colonne di peso e altezza verranno sostituite da questo valore.
       - data: dataset (DataFrame)
       - colonna_peso: nome colonna peso (string)
       - colonna_alt: nome colonna altezza (string)
       - nome_colonna_bmi: nome nuova colonna"""
    data[colonna_peso] = [float((p.replace(",", "."))) if not isinstance(p, float) else p for p in data[colonna_peso]]
    data[colonna_alt] = [float(p) if not isinstance(p, float) else p for p in data[colonna_alt]]
    data[nome_colonna_bmi] = data[colonna_peso] / ((data[colonna_alt] / 100) ** 2)
    data[nome_colonna_bmi] = [float(bmi) for bmi in data[nome_colonna_bmi]]
    data.drop(colonna_peso, axis=1, inplace=True)
    data.drop(colonna_alt, axis=1, inplace=True)
    
    return data

In [55]:
cols_peso = ['somm'+str(i)+'_peso' for i in range(1,7)]
cols_altezza = ['somm'+str(i)+'_altezza' for i in range(1,7)]
for i in range(len(cols_peso)):
    df = calcola_BMI(df, cols_peso[i], cols_altezza[i], 'somm'+str(i+1)+'_BMI')

In [56]:
cols_pes = ['somm'+str(i)+'_peso' for i in range(1,7)]
cols_alt = ['somm'+str(i)+'_altezza' for i in range(1,7)]
cols_num = set(cols_num) - set(cols_pes+cols_alt)

***
### Gestione dati numerici nulli

In [57]:
for col in cols_num:
    df[col] = [float(p) for p in df[col]]
    mean = np.mean(df[col].dropna())
    df.fillna(mean, inplace=True)

In [58]:
cols_null = [col for col in df.columns if df[col].isnull().sum() > 0]
cols_null

[]

***
### Rimozione colonne relative alle somministrazioni ad eccezione di quella del primo BMI

In [59]:
# rimozione OPZIONALE delle colonne relative alle somministrazioni
cols = [col for col in df.columns if 'somm' in col]
cols = set(cols) - set(['somm1_bmi'])
#df.drop(cols, axis=1, inplace=True)

***
### Rimozione degli outlier

In [60]:
threshold = 3
outliers_index = []
tot_sum = 0
count = 0
for col in cols_num:
    df[col] = [float(p) for p in df[col]]
    mean = np.mean(df[col])
    std = np.std(df[col])
    for i,x in enumerate(df[col]):
        z = (x - mean) / std
        if np.abs(z) > threshold:
            outliers_index.append(i)

df.drop(outliers_index, inplace=True)
df.shape

(312, 76)

***
### Esportazione dei dati:

In [61]:
def esporta_dati(data, path):
    """Funzione per il salvataggio del dataset in un file csv.
       - data: dataset (DataFrame)
       - path: percorso in cui salvare i dati """ 
    data.to_csv(path)

In [62]:
esporta_dati(df, '/Users/eliaceccolini/Documents/Uni/Tesi/Dataset_finale/Datasets/'+export_name+'.csv')